# **Classificação**

In [1]:
# importe as principais bibliotecas
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

## **TOC:**
Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) [Introdução](#intro)
- 2) [Métricas de performance para problemas de classificação](#metricas)
- 3) [Regressão logística](#reglog)
    - 3.1) [Teoria](#teoria_reglog)
    - 3.2) [Exemplo](#exemplo_reglog)
    - 3.3) [ROC](#roc)
    - 3.4) [Precision vs Recall](#precision_recall)
- 4) [Exercicio](#exercicios)

____
____
____

## 1) **Introdução** <a class="anchor" id="nao_supervisionado"></a>

**Problemas de Classificação** são aqueles em que queremos determinar a que **CATEGORIA** dentro de um **CONJUNTO DE CATEGORIAS** uma dada observação pertence, com base em suas features.

Para isso, construímos um **CLASSIFICADOR**: modelo que tem como input as features (contínuas ou discretas) e como output uma entre as classes (discretas)


Principal diferença entre problemas de regressão e classificação:
- Regressão: valores contínuos;
- Classificação: valores (classes) discretas (binárias ou não).

<center><img src="https://i0.wp.com/vinodsblog.com/wp-content/uploads/2018/11/Classification-vs-Regression.png?fit=2048%2C1158&ssl=1" width=700></center>

---

<center><img src="https://i.pinimg.com/originals/71/8e/6a/718e6a40e1782bead960e58d3c52663b.png" width=300></center>

Exemplos de problemas de classificação:
- Detecção de e-mails SPAM: um e-mail é SPAM ou não?;
    - Features: palavras contidas no corpo do e-mail; remetente; assunto;
- Detecção de doenças: que codição médica a pessoa tem?
    - Features: sintomas fisiológicos; resultados de exames (medidas de variáveis biológicas);
- Detecção do tipo de documento: secreto, confidencial ou não-sensível?
    - Features: palavras no corpo do texto; título;
- Detecção de fraudes de cartão de crédito: uma operação é fraudulenta ou não?;
    - Features: histórico de transações; hora, local e frequência das transações; tipo de compra;
- Modelo de risco de crédito: qual é a chance de determinada pessoa não pagar seu empréstimo?
    - Features: histórico de pagamento; score de crédito;
    
  
<center><img src="https://developers.google.com/machine-learning/guides/text-classification/images/TextClassificationExample.png" width=500></center>



Veremos hoje um dos mais simples e poderosos classificadores: a **LOGIT!**


---

## 2) **Métricas de performance para problemas de classificação** <a class="anchor" id="metricas"></a>

Após treinar o modelo, como podemos avaliar sua performance?

No caso de problemas de classificação, existem **métricas específicas**, e também um importante conceito chamado de **Matriz de Confusão**.

A **matriz de confusão** leva em consideração as **classes preditas** e as **classes verdadeiras** da base de **teste**, e contabiliza a performance do modelo:

<center><img src=https://diegonogare.net/wp-content/uploads/2020/04/matrizConfusao-600x381.png height="400" width="400"></center>

No Sklearn, a notação muda um pouco:

<center><img src="https://static.packt-cdn.com/products/9781838555078/graphics/C13314_06_05.jpg" width=400></center>

Note que a diagonal principal são as observações que o modelo acertou! Temos:

- Verdadeiros Positivos (VP): classificação correta da classe positivo;
- Verdadeiros Negativos (VN): classificação correta da classe negativo;
- Falsos Positivos (FP, erro tipo I): correto: negativo. Previsto: positivo.
- Falsos Negativos (FN, erro tipo II): correto: positivo. Previsto: negativo.

Um jeito fácil de lembrar os tipos de erros:


<center><img src="https://i.pinimg.com/originals/f6/9b/11/f69b111014ef466fe541a393346d2c3a.jpg" height="200" width="300"></center>


Além disso, temos as seguintes métricas numéricas de avaliação:

- **Acurácia (Accuracy)**: porcentagem de classificações CORRETAS do modelo;

- **Precisão (Precision)**: das respostas retornadas, quantas são relevantes? -- é a razão entre verdadeiros positivos e o  número de **preditos positivos**, isto é, positivos quanto à **label predita pelo modelo**.

- **Revocação/Sensibilidade (Recall/Sensitivity)**: das respostas relevantes, quantas são retornadas? -- é a razão entre verdadeiros positivos e o  número de **verdadeiramente positivos**, isto é, positivos quanto à **label real**.

- **F1-Score**: média harmônica de precision e recall.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/1200px-Precisionrecall.svg.png" width=250></center>

Devido ao <a href="https://medium.com/opex-analytics/why-you-need-to-understand-the-trade-off-between-precision-and-recall-525a33919942">tradeoff entre precision e recall</a>, a métrica a ser otimizada é o F1! 


<center><img src="https://miro.medium.com/max/1080/1*t1vf-ofJrJqtmam0KSn3EQ.png" height="300" width="300"></center>

Adiante, veremos como calcular a matriz de confusão e as métricas acima para problemas de classificação!

___

## 3) **Regressão Logística** <a class="anchor" id="reglog"></a>

### 3.1) **Teoria** <a class="anchor" id="teoria_reglog"></a>

A [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) (também chamado de **logit**), apesar do nome, é um modelo de classificação!

- Classificação binária: duas classes (0 e 1);
- Classificação multiclasse: n classes (0, 1, ..., n-1), com $n \in \mathbb{N}$

Dado um conjunto de features $x$ qual a probabilidade de $x \in 1$, queremos encontrar um modelo que nos dê:

$$ P( x \in 1 | x) $$

Naturalmente, $0 \le P(x) \le 1$. Assim, por exemplo, se:
- $P(x) \ge 0,5$: x pertence à classe 1
- $P(x) < 0.5$: x pertence à classe 0

Obs.: este valor de 0.5 (50%) é chamado de "cutoff", e pode ser ajustado, embora seja comum fixá-lo em 50%!

Poderíamos pensar em utilizar a regressão linear em nossos problemas de classificação, mas isso não é uma boa ideia: acabamos encontrando probabilidades negativas e fit ruim!

No exemplo a seguir, temos a probabilidade de não-pagamento (default) de um empréstimo com base em uma feature (balanço). Note probabilidades negativas!

<figure>
    <center>
    <img src="https://s3-sa-east-1.amazonaws.com/lcpi/70189f79-2886-4e59-893b-1dac9dd64078.png" height="400" width="400">
    <figcaption>
        Regressão Linear para classificação. 
    </figcaption>
    </center>
</figure> 

Para resolver este problema, podemos adaptar a função de regressão linear para uma função que tem imagem entre 0 e 1. Seria legal se tivéssemos algo como:

<figure>
    <center>
    <img src="https://s3-sa-east-1.amazonaws.com/lcpi/6d54529a-d295-47a3-8a11-1f426fde7229.png" height="400" width="400">
    </center>
</figure> 

Um exemplo de tal função é a FUNÇÃO LOGÍSTICA ou FUNÇÃO SIGMOIDAL:

<center><img src="https://miro.medium.com/max/970/1*Xu7B5y9gp0iL5ooBj7LtWw.png" width=400></center>

Note que:
- $z \in \mathbb{R}$
- $0 \le \phi(z) \le 1$

Para incorporar a ideia da regressão linear na regressão logística, tomamos:

- $z = \beta_0 + \beta_1x$, que é o modelo de regressão linear (uma variável);

E substituímos na função logística:

$$\phi(x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}}$$

Com isso, tomamos qualquer output real do modelo linear e transformamos em um valor entre 0 e 1, como queríamos!

No nosso caso, como queremos modelar probabilidades, tomamos, no caso de uma feature:

$$P(x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}}$$

Ou, para a regressão logística múltipla com $p$ features:


$$P((x_1, \cdots, x_p)) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \cdots + \beta_p x_p)}}$$

---

## **Exemplo** <a class="anchor" id="exemplo_reglog"></a>

Para introduzirmos as ideias, utilizaremos um dataset de marketing (propagandas/advertising). Este é um dataset artificial e didático, com os dados bem separáveis, o que é ótimo para ilustração!

A base que utilizaremos contém as seguintes colunas:

* 'Daily Time Spent on Site': tempo que o cliente ficou no site (em minutos);
* 'Age': idade do cliente (em anos);
* 'Area Income': média salarial (por ano) da região geográfica do cliente;
* 'Daily Internet Usage': tempo médio (em minutos) que o cliente fica na internet;
* 'Ad Topic Line': título do anúncio;
* 'City': cidade do cliente;
* 'Male': dummy indicando se o cliente é do sexo masculino (1) ou não (0);
* 'Country': país do cliente;
* 'Timestamp': marcação de tempo em que o cliente clickou no anúncio OU fechou a página
* 'Clicked on Ad': dummy indicando se o cliente clickou no anúncio (1) ou não (0).

Nosso objetivo é criar um modelo que possa prever se um determinado usuário clickará em um anúncio online ou não, com base em suas características pessoais/comportamentais, bem como informações relativas ao anúncio.

Tomamos como variáveis independentes (preditores/features) as primeiras 9 colunas, enquanto nossa variável dependente (target) é a última coluna ("Clicked on Ad").

Ou seja, nosso modelo deve ser capaz de dizer se um usuário com um conjunto particular das 9 features clickará no anúncio ou não. 

__IMPORTANTE!__

Pense no problema de negócio que estamos querendo resolver com nosso modelo -- direcionamento de marketing! Temos os dados dos nossos clientes (customer-centric), nós os conhecemos! Não podemos utilizar essa informação a nosso favor?

Talvez não faça sentido exibir o anúncio para um usuário que tem baixa probabilidade de clickar no ad, não é mesmo? 

Por outro lado, é muito mais eficiente direcionar nosso marketing aos clientes com alta chance de clickar no nosso anúncio!

Assim, economizamos dinheiro (todo anúncio é pago!), e ganhamos em eficiência e alcance!

---

O código abaixo é apenas para formatar os números em até 3 casas decimais. 

Fica aqui pra conhecimento e também pq vai nos auxiliar a ver melhor as probabilidades no final.

Alguma observação notável?

Temos um dataset balanceado no target, o que __bem raro na vida real!__

Um dataset desbalanceado pode causar sérios problemas de performance ao modelo! Há várias técnicas para lidar com tal problema, mas, neste primeiro exemplo, não nos preocuparemos com isso...

Como tínhamos comentado no início, nossos dados são muito bem separáveis!

Isto favorece bastante a performance do nosso modelo. Mas, lembre-se, é bem raro encontrar casos assim na vida real! (É aí que devemos partir para métodos mais avançados, como SVM, árvores, etc.)

Vamos começar a construir o modelo?

__Modelo treinado!__

Vamos ver os coeficientes do modelo:

Lembre-se que, diferentemente da regressão linear, devido ao fato da função logística ser uma exponencial, a variação de $P(x)$ depende de x, e não apenas dos coeficientes! Então, a interpretação dos coeficientes não é tão imediata. 

Mas, os sinais carregam significado. Para um coeficiente:
- positivo ($\beta_i > 0$), temos que um aumento em x levará a um aumento de $P(x)$;
- negativo ($\beta_i < 0$), temos que um aumento em x levará a uma diminuição de $P(x)$

Mas, a variacão de $P(x)$ em si, depende do valor de x!

__Agora que o modelo está treinado, vamos avaliá-lo!__

Como vimos no passo 2, em problemas de classificação é muito comum utilizarmos a **matriz de confusão** e as **métricas de classificação** para avaliar nossos modelos.

Dado isso, o sklearn já disponibilica estas funcionalidades:

Conforme esperado, nosso modelo está muito bom! Um f1-score tão alto na vida real é algo notável!

Isso se deve à grande separabilidade dos nossos dados!

Além dos coeficientes do modelo, algo muito interessante que a classe do sklearn tem é o método `predict_proba()`

Esse método retorna exatamente qual é a probabilidade modelada pelo logit.

Isso pode ser muito útil, pois assim conseguimos **mudar qual é o threshold de esclha de classe** para ser algo diferente de 0.5!


Juntando todas as etapas:

### 3.3) **ROC** <a class="anchor" id="roc"></a>

### 3.4) **Precision vs Recall** <a class="anchor" id="precision_recall"></a> 

___

## 4) **Exercícios** <a class="anchor" id="exercicios"></a>

Vamos treinar mais? Agora é com você!

### **Exercício 1**: Construa um classificador usando regressão logística (titanic)

In [2]:
# APENAS DADOS NUMÉRICOS

# leia a base

# dropando as linhas com NaN

# separe as features e o target
# apenas features numéricas!!

# importe a classe do classificador

# instancie a classe

# faça o train-test split

# treine o modelo

# dê uma olhada nos coeficientes

# dê uma olhada nas classes do modelo

# faça previsões

# dê uma olhada nas probabilidades das previsões

# avalie o modelo



____________

### Exercício 2: Construa um classificador usando regressão logística (iris)

In [3]:
# leia a base

# separe as features e o target

# importe a classe do classificador

# instancie a classe

# faça o train-test split
    
# treine o modelo

# dê uma olhada nos coeficientes

# dê uma olhada nas classes do modelo

# faça previsões

# dê uma olhada nas probabilidades das previsões

# avalie o modelo


O problema do dataset iris é um problema de **classificação multiclasse**, pois há mais do que duas classes a serem preditas: no caso, 3.

<center><img src="https://utkuufuk.com/2018/06/03/one-vs-all-classification/one-vs-all.png"></center>

Como vimos acima, o operacional de construção do modelo muda em absolutamente **nada**.

No entanto, conceitualmente, há algumas mudanças: a rigor, o modelo passa a se chamar **regresão logística MULTINOMIAL**, cujo processo de classificação é dado pela função **softmax**:

<center><img src="https://i.stack.imgur.com/YLeRi.png" width=600></center>

Para quem quiser saber mais sobre o "logit score", [clique aqui](https://stats.stackexchange.com/questions/329857/what-is-the-difference-between-decision-function-predict-proba-and-predict-fun).
